In [ ]:
'''Comparison'''

import pandas as pd
import numpy as np
import sys, os, pathlib
import joblib 
import time 


NOTEBOOK_DIR = pathlib.Path(os.getcwd())  
ROOT = NOTEBOOK_DIR.parents[0]  

sys.path.insert(0, str(ROOT))

from src.dataset_generator import sample_heston_lhs
from src.Heston_FFT import heston_price_fft
from src.Heston_MC import heston_mc_qe_price
heston_nn = joblib.load("model_ann_3.pkl")


# FFT params
alpha  = 1.5
fft_N  = 4096     
eta    = 0.1      

# MC params
mc_steps = 150
mc_paths = 100_000
seed     = 42

# Random dataset (LHT)
test_df = sample_heston_lhs(100, seed=123, S0=105, q=0)
test_df = pd.DataFrame(test_df)


# Solver
def solver(df, alpha, eta, N, steps, path):

    S       = df['S0']
    K       = df['K']
    T       = df['T']
    r       = df['r']
    v0      = df['v0']
    kappa   = df['kappa']
    theta   = df['theta']
    sigma_v = df['sigma_v']
    rho     = df['rho']
    q       = df['q']

    # results
    fft_price, fft_time = [], []
    mc_price,  mc_time  = [], []
    nn_price,  nn_time  = [], []

    # loop 
    for i, row in df.iterrows():
        Si, Ki, Ti = float(row['S0']), float(row['K']), float(row['T'])
        ri, v0i    = float(row['r']), float(row['v0'])
        kappai, thetai = float(row['kappa']), float(row['theta'])
        sigi, rhoi = float(row['sigma_v']), float(row['rho'])
        qi = float(row['q'])

        # FFT
        t0 = time.perf_counter()
        p_fft = heston_price_fft(
            S=Si, K=Ki, T=Ti, r=ri, v0=v0i,
            kappa=kappai, theta=thetai, sigma_v=sigi, rho=rhoi,
            option_type="call", q=qi, alpha=alpha, N=N, eta=eta
        )
        t1 = time.perf_counter()

        # MC 
        t2 = time.perf_counter()
        p_mc = heston_mc_qe_price(
            Si, Ki, Ti, ri, qi,
            v0i, kappai, thetai, sigi, rhoi,
            steps=steps, paths=path,
            antithetic=True, seed=None,
            option_type="call"
        )
        t3 = time.perf_counter()
        if isinstance(p_mc, tuple):
            p_mc = p_mc[0]

        # NN
        feats = ["m", "T", "r", "v0", "kappa", "theta", "sigma_v", "rho"]
        x = np.array([[row[f] for f in feats]], dtype=float)
        t4 = time.perf_counter()
        p_nn = float(heston_nn.predict(x)[0])
        t5 = time.perf_counter()

        # push risultati
        fft_price.append(float(p_fft))
        fft_time.append(t1 - t0)

        mc_price.append(float(p_mc))
        mc_time.append(t3 - t2)

        nn_price.append(float(p_nn))
        nn_time.append(t5 - t4)

    # cols
    df['FFT_price'] = fft_price
    df['FFT_time']  = fft_time

    df['MC_price']  = mc_price      
    df['MC_time']   = mc_time

    df['MC_error']  = abs(df['MC_price'] - df['FFT_price'])

    df['NN_price']  = nn_price
    df['NN_time']   = nn_time
    df['NN_error']  = abs(df['NN_price'] - df['FFT_price'])

    return df


results_df = solver(test_df, alpha, eta, fft_N, mc_steps, mc_paths)
#results_df.to_csv('data/results_df.csv', index=False)
results_df

c:\Users\giovanni.piva\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\giovanni.piva\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\giovanni.piva\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\giovanni.piva\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\giovanni.piva\AppData\Local\Programs\Python\Python312\Lib\site-pack

,S0,K,m,T,r,q,v0,kappa,theta,sigma_v,rho,FFT_price,FFT_time,MC_price,MC_time,MC_error,NN_price,NN_time,NN_error
0,105,115.158920,1.096752,0.811002,0.092148,0,0.076527,4.841289,0.071912,0.418670,-0.177248,9.209616,0.001964,9.231550,1.724215,0.021934,9.222542,0.000588,0.012925
1,105,117.510220,1.119145,0.969393,0.033654,0,0.014872,1.874977,0.087795,0.283398,-0.779253,5.532732,0.001620,5.522624,1.429417,0.010109,5.638286,0.000896,0.105554
2,105,100.730320,0.959336,1.141289,0.032030,0,0.059229,4.702282,0.064080,0.354168,-0.882984,15.253949,0.002257,15.236218,1.416022,0.017731,14.914779,0.000521,0.339170
3,105,105.577276,1.005498,0.535943,0.073124,0,0.038726,2.389665,0.085412,0.168723,-0.477904,9.194507,0.001093,9.179933,1.354834,0.014573,8.909756,0.000491,0.284750
4,105,135.499952,1.290476,1.206278,0.050265,0,0.059001,4.854463,0.055949,0.413841,-0.117014,3.819320,0.001113,3.812203,1.354202,0.007117,3.917590,0.000826,0.098270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,105,123.320228,1.174478,0.218166,0.005058,0,0.021303,3.953465,0.078413,0.724006,-0.866921,0.002820,0.001219,0.002287,1.330589,0.000533,0.260496,0.000504,0.257676
96,105,115.846338,1.103298,0.142529,0.010206,0,0.030798,4.693649,0.011166,0.257579,-0.106894,0.152711,0.001077,0.146084,1.312832,0.006628,0.735906,0.000582,0.583195
97,105,105.095538,1.000910,0.550792,-0.007123,0,0.026378,4.117916,0.063895,0.453379,-0.926720,6.322990,0.001240,6.349534,1.328385,0.026544,6.152672,0.000533,0.170318
98,105,97.119062,0.924943,1.159459,0.035700,0,0.016154,4.573358,0.020382,0.285945,-0.474225,13.817800,0.001139,13.828410,1.308663,0.010610,14.129825,0.000578,0.312025


In [34]:
# check

fft_try =  heston_price_fft(S=105, K=115.159, T=0.811, r=0.0921, v0=0.0765, kappa=4.841, theta=0.072, sigma_v=0.419, rho=-0.177,
                             option_type="call", q=0, alpha=alpha, N=fft_N, eta=eta)
MC_try =  heston_mc_qe_price(S0=105,  K=115.159, T=0.811, r=0.0921, q=0, v0=0.0765, kappa=4.841, theta=0.072, sigma_v=0.419, rho=-0.177,
                                         steps=mc_steps, paths=mc_paths,
                                         antithetic=True, seed=seed, option_type="call")
df_NN_try = pd.DataFrame([{"m": 1.0967,"T": 0.81,"r": 0.092,"v0": 0.076,"kappa": 4.841,"theta": 0.072,"sigma_v": 0.419,"rho": -0.177}])
NN_try = heston_nn.predict(df_NN_try)[0]

print(f"fft: {fft_try:.3f}")
print(f"mc: {MC_try[0]:.3f}")
print(f"nn: {NN_try:.3f}")

fft: 9.212
mc: 9.239
nn: 9.203


In [35]:
fft_avg_time = results_df['FFT_time'].mean()
mc_avg_time = results_df['MC_time'].mean()
nn_avg_time = results_df['NN_time'].mean()

fft_std_time = results_df['FFT_time'].std()
mc_std_time  = results_df['MC_time'].std()
nn_std_time  = results_df['NN_time'].std()

fft_total_time = results_df['FFT_time'].sum()
mc_total_time  = results_df['MC_time'].sum()
nn_total_time  = results_df['NN_time'].sum()


fft_avg_error = results_df['MC_error'].mean()
NN_avg_error = results_df['NN_error'].mean()

mc_std_error = results_df['MC_error'].std()
nn_std_error = results_df['NN_error'].std()

mc_max_error = results_df['MC_error'].max()
nn_max_error = results_df['NN_error'].max()


summary = pd.DataFrame({
    "avg_time":   [fft_avg_time, mc_avg_time, nn_avg_time],
    "std_time":   [fft_std_time, mc_std_time, nn_std_time],
    "avg_error":  [0, fft_avg_error, NN_avg_error],   # FFT baseline = 0
    "std_error":  [0, mc_std_error, nn_std_error],
    "max_error":  [0, mc_max_error, nn_max_error]
}, index=["FFT", "MC", "NN"])

summary

,avg_time,std_time,avg_error,std_error,max_error
FFT,0.001321,0.000322,0.000000,0.000000,0.000000
MC,1.396472,0.141282,0.024677,0.033218,0.193314
NN,0.000672,0.000240,0.236186,0.181621,0.859916
